In [ ]:
!pip install mrjob

In [11]:
%%file hw1_mrA_337604821_326922390.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

def split_properley (line):
    item = line.split(',')
    date = item[-3] # taking third item from the righ to left on the row, which represents the date
    air_time = int(item[-2]) if item[-2].isnumeric() else 0 #getting the integer value fo the item second from the right side of the row

    if item[2] != item[-4]:# checking if there are multiple genres or only 1.
        genres = line.split('\"')[1].split(',') if item[0] != 'title' else "Hello World"
    else:# There is only one genre for this tuple
      genres = [item[2]]
    title = item[0].strip()
    return title, genres, air_time, date

class MRWordFrequencyCount(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_count_genres),
            #MRStep(reducer=self.reducer_max)
        ]

    def mapper(self, _, line):
        title, genres, air_time, date = split_properley(line)# get relevant data from the tuple
        try:
            if 70000 <= int(air_time) < 90000 and re.search('[jqz]', title.lower()):# filter out by airtime and that title contains one of the letters j,q,z
                for genre in genres:
                    if genre.strip() in ['Sitcom', 'Talk', 'Politics', 'Spanish', 'Community', 'Martial arts']:
                        ls = [title]
                        ls.extend(genres)
                        yield (tuple(ls), date)# if one of the genres is in the list then we will move this tuple reformatted to the next step
        except Exception as e:
            pass

    def reducer(self, key, dates):
        yield key, len(set(dates))# reduce the value so it contains the number of unique dates

    def reducer_count_genres(self, key, values):
        total_dates = 0
        for dates_count in values:
            total_dates += dates_count
        yield key, (total_dates, len(key)-1)# counting number of dates per key, counting number of genres



if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting hw1_mrA_337604821_326922390.py


In [12]:
!python hw1_mrA_337604821_326922390.py 420k_daily_prog_data.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/hw1_mrA_337604821_326922390.root.20240710.125931.013212
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/hw1_mrA_337604821_326922390.root.20240710.125931.013212/output
Streaming final output from /tmp/hw1_mrA_337604821_326922390.root.20240710.125931.013212/output...
["El Joven Ju\u00e1rez", "Spanish", "Biography"]	[1, 2]
["El Joven del Carrito", "Spanish", "Comedy"]	[1, 2]
["El Oreja Rajada", "Spanish", "Drama"]	[1, 2]
["El Palenque", "Talk"]	[1, 1]
["El Rediezcubrimiento de M\u00e9xico", "Spanish", "Comedy-drama"]	[1, 2]
["El Santos vs la Tetona Mendoza", "Spanish", "Comedy", "Animated"]	[1, 3]
["El Santos vs. la T...a Mendoza", "Spanish", "Comedy", "Animated"]	[1, 3]
["El Tejedor de Milagros", "Spanish", "Drama"]	[1, 2]
["El Tigre de Guanajuato", "Spanish", "Adventure"]	[1, 2]
["El Vizconde de Montecristo", "Spanish", "Comedy"]	[1, 2]
["El mejor"

------------------------------MarkDown --------------------------------------
Part A Q1.B

In [13]:
%%file hw1_mrB_337604821_326922390.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re

def split_properley (line):
    item = line.split(',')
    date = item[-3]
    air_time = int(item[-2]) if item[-2].isnumeric() else 0

    if item[2] != item[-4]:
        genres = line.split('\"')[1].split(',') if item[0] != 'title' else "Hello World"
    else:
      genres = [item[2]]
    title = item[0].strip()
    return title, genres, air_time, date

class MRWordFrequencyCount(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_count_genres),
            MRStep(reducer=self.reducer_max)
        ]

    def mapper(self, _, line): 
        title, genres, air_time, date = split_properley(line)
        try:
            if 70000 <= int(air_time) < 90000 and re.search('[jqz]', title.lower()):
                for genre in genres:
                    if genre.strip() in ['Sitcom', 'Talk', 'Politics', 'Spanish', 'Community', 'Martial arts']:
                        ls = [title]
                        ls.extend(genres)
                        yield (tuple(ls), date)
        except Exception as e:
            pass

    def reducer(self, key, dates):
        unique_dates = set(dates)
        yield key, len(unique_dates)

    def reducer_count_genres(self, key, values):
        total_dates = 0
        for dates_count in values:
            total_dates += dates_count
        yield None, (key, total_dates + len(key)-1)# yield key with the sum of genres and total unique dates

    def reducer_max(self, _, values):
        yield max(values, key=lambda x: x[1])



if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting hw1_mrB_337604821_326922390.py


In [14]:
!python hw1_mrB_337604821_326922390.py 420k_daily_prog_data.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/hw1_mrB_337604821_326922390.root.20240710.130008.257936
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in /tmp/hw1_mrB_337604821_326922390.root.20240710.130008.257936/output
Streaming final output from /tmp/hw1_mrB_337604821_326922390.root.20240710.130008.257936/output...
["The King of Queens", "Sitcom"]	147
Removing temp directory /tmp/hw1_mrB_337604821_326922390.root.20240710.130008.257936...


------------------------------MarkDown --------------------------------------

Part **B**

In [ ]:
# Install PySpark on the Colab machine - code in "חומר עזר קולאב"
# Cell 1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
# Cell 2
!pip install --force-reinstall pyspark==3.4
!pip install findspark

In [ ]:
# Install PySpark on the Colab machine - code in "חומר עזר קולאב"
# Cell 1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
# Cell 2
!pip install --force-reinstall pyspark==3.4
!pip install findspark

In [32]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs
from pyspark.sql.types import*
from pyspark.sql.functions import to_date, dayofweek, date_format
import logging
import re

In [4]:
# Before getting/creating the Session, we can try to modify parameters.
spark = SparkSession.builder.appName('Spark - HW01 Q2')\
    .getOrCreate()
sc = spark.sparkContext
# keep only important logs
spark.sparkContext.setLogLevel("ERROR")

In [52]:
bl = ["Big", "the", "bang", "theory", "almanac", "met", "mother", "your", "city", "anatomy", "game", "throne", "guy", "family", "friend", "senate", "two"]

g = lambda genre, typ, pnts: pnts if (typ in genre and len(genre) == 1) else 0 # give pnts based on matching genre if it's the only genre in tuple
time_len = lambda time: time / 15
def white_list(ls):
  for word in bl:# check if given ls has a word in the black list, false if it does otherwise True.
    for wrd in re.split('[,:]?\s', ls):
      if word.lower() == wrd.lower():
        return False
  return True
# score calculation of tuple
score_func = lambda title, genres, date, duration: (date + sum(g(genres, gg, p) for gg, p in zip(["Action", "Documentary", "Sitcom"],[90,90,7])) + time_len(duration)) if white_list(title) else 0

In [61]:
def split_properly(item):
  date = 3 if item[-3] and item[-3] in [1,3,5, 7] else 0 #looking for odd date
  genres_tuple = tuple(item[2].split(',')) if item[2] else ()# find the different genres
  title = item[0]
  duration = float(str(item[-1])) if re.match(r"^-?[0-9]+(\.[0-9]+)?$", str(item[-1])) else 0 # check if format matches and convert to float
  return title, genres_tuple, date, duration

In [62]:
df = spark.read.csv('420k_daily_prog_data.csv', header=True, inferSchema=True)
df = df.withColumn("air_date", dayofweek(to_date(df.air_date, "yyyyMMdd")))# convert date to corresponding weekday value

In [63]:
sol = df.rdd.map(split_properly) \
          .map(lambda row: ((row[0], row[1]), score_func(row[0], row[1], row[2], row[3]))) \
          .reduceByKey(lambda x, y: x + y) \
          .sortBy(lambda x: x[1], ascending=False)
# rdd that applies split_properly on each row, then maps the title and genres to a score on each tuple
# we then reduce to count duplicate keys to get total score and sort by to find the top ones

In [64]:
solution = sol.collect()[:20]
# run the action and take the top 20

In [65]:
for (title, genres), score in solution:
    genres_str = ', '.join(genres)
    print(f'{{"{title}", {genres_str}}} | {score}')

{"SIGN OFF", Special} | 22648.13333333333
{"Everybody Loves Raymond", Sitcom} | 14961.533333333326
{"Documentary", Documentary} | 13933.06666666666
{"Mike & Molly", Sitcom} | 13269.600000000002
{"Hot in Cleveland", Sitcom} | 11855.666666666668
{"Seinfeld", Sitcom} | 11084.133333333335
{"Community", Sitcom} | 9085.533333333333
{"ABC World News Now", News} | 8337.999999999989
{"Strange Inheritance", Documentary} | 8113.0
{"Un Mundo Maravilloso", Documentary} | 8078.0
{"Drug Wars", Documentary} | 7890.0
{"Weather Radar", Weather} | 6987.0
{"True Life", Documentary} | 5923.466666666664
{"Larry King Special Report", Consumer} | 5796.4000000000015
{"Classic Arts Showcase", Art} | 5731.666666666666
{"Programa Pagado", Shopping} | 5447.799999999999
{"Live Cameras and Forecast Maps", Weather} | 5391.733333333235
{"Forensic Files", Reality, Crime} | 5277.0
{"Local Weather", Weather} | 5257.0
{"Dateline on ID", Documentary} | 5254.0
